In [ ]:
using Pkg
Pkg.status()
Pkg.resolve()
Pkg.update()
Pkg.instantiate()

In [ ]:
using SeismicWaves
using LinearAlgebra

###################################################################
using Logging
using Plots, Plots.Measures


error_logger = ConsoleLogger(stderr, Logging.Info)

function gaussian_vel_1D(nx, c0, c0max, r, origin=(nx+1)/2)
    sigma = r / 3
    amp = c0max - c0
    f(x) = amp * exp(-(0.5 * (x - origin)^2 / sigma^2))
    return c0 .+ [f(x) for x in 1:nx]
end

function gaussian_vel_2D(nx, ny, c0, c0max, r, origin=[(nx+1)/2, (ny+1)/2])
    sigma = r / 3
    amp = c0max - c0
    f(x,y) = amp * exp(-(0.5 * ((x - origin[1])^2 + (y - origin[2])^2) / sigma^2))
    return c0 .+ [f(x,y) for x in 1:nx, y in 1:ny]
end

function gaussian_vel_3D(nx, ny, nz, c0, c0max, r, origin=[(nx+1)/2, (ny+1)/2, (nz+1)/2])
    sigma = r / 3
    amp = c0max - c0
    f(x,y,z) = amp * exp(-(0.5 * ((x - origin[1])^2 + (y - origin[2])^2 + (z - origin[3])^2) / sigma^2))
    return c0 .+ [f(x,y,z) for x in 1:nx, y in 1:ny, z in 1:nz]
end

function disc_vel_1D(nx, c0, c0max, r, origin=(nx+1)/2)
    vel = c0 .* ones(nx)
    for i in 1:nx
        if norm(origin - i) <= r
            vel[i] = c0max
        end
    end
    return vel
end

function disc_vel_2D(nx, ny, c0, c0max, r, origin=[(nx+1)/2, (ny+1)/2])
    vel = c0 .* ones(nx, ny)
    for i in 1:nx
        for j in 1:ny
            if norm(origin .- [i, j]) <= r
                vel[i,j] = c0max
            end
        end
    end
    return vel
end

function disc_vel_3D(nx, ny, nz, c0, c0max, r, origin=[(nx+1)/2, (ny+1)/2, (nz+1)/2])
    vel = c0 .* ones(nx, ny, nz)
    for i in 1:nx
        for j in 1:ny
            for k in 1:nz
                if norm(origin .- [i, j, k]) <= r
                    vel[i,j,k] = c0max
                end
            end
        end
    end
    return vel
end

function plot_nice_heatmap(A; lx=size(A,1), ly=size(A,2), step=1, shift=0)
    heatmap(0-shift:step:lx-shift, 0-shift:step:ly-shift, A';
            aspect_ratio=:equal, margin=5mm, cmap=:jet1, yflip=true, xlims=(0-shift,lx-shift), ylims=(0-shift,ly-shift))

end

function plot_nice_heatmap_grad(A; lx=size(A,1), ly=size(A,2), step=1, shift=0)
    vmax = maximum(abs.(A))
    heatmap(0-shift:step:lx-shift, 0-shift:step:ly-shift, A';
            aspect_ratio=:equal, margin=5mm, cmap=:RdBu, yflip=true, xlims=(0-shift,lx-shift), ylims=(0-shift,ly-shift),
            clims=(-vmax, vmax))

end

In [ ]:
# time stuff
nt = 1000
c0 = 1000
c0max = 1300
r = 35
dh = dx = dy = 5
dt = dh / sqrt(2) / c0max
halo = 20
rcoef = 0.0001
times = collect(Float64, range(0.0; step=dt, length=nt)) # seconds

##========================================
# create a velocity model
nx = 201
ny = 201
lx = (nx-1) * dx
ly = (ny-1) * dy
matprop_const = VpAcousticCDMaterialProperty(c0 .* ones(nx, ny))
# gaussian perturbed model
matprop_gauss = VpAcousticCDMaterialProperty(gaussian_vel_2D(nx, ny, c0, c0max, r))

display(plot_nice_heatmap(matprop_const.vp; lx=nx*dx, ly=ny*dy, step=dh))
p = plot_nice_heatmap(matprop_gauss.vp; lx=nx*dx, ly=ny*dy, step=dh)
display(p)

In [ ]:
# shots definition
nshots = 20
shots = Vector{Shot{Float64}}()  #Pair{Sources, Receivers}}()
# source time function
f0 = 10
t0 = 4 / f0

for i in 1:nshots
    # sources definition
    nsrcs = nshots
    distsrcs = 30
    xsrcs = (lx/2) .- distsrcs .* ((nsrcs+1)/2 .- collect(1:nsrcs))
    ysrc  = (halo + 10) * dy
    possrcs = reshape([xsrcs[i], ysrc], 1, 2)
    srcs = Sources(
        possrcs,
        repeat(rickersource1D.(times, 2/f0, f0), nt, 1),
        f0
    )

    # receivers definition
    nrecs = 20
    distrecs = 30
    xrecs = (lx/2) .- distrecs .* ((nrecs+1)/2 .- collect(1:nrecs))
    yrec  = ly - (halo + 10) * dy
    posrecs = hcat(xrecs, fill(yrec, nrecs))
    recs = Receivers(
        posrecs,
        nt
    )

    # add pair as shot
    push!(shots, Shot(; srcs=srcs, recs=recs)) # srcs => recs)

    # display shot
    p = plot_nice_heatmap(matprop_gauss.vp; lx=nx*dh, ly=ny*dh, step=dh)
    # display rays
    for irec in 1:nrecs
        plot!([possrcs[1,1], posrecs[irec,1]], [possrcs[1,2], posrecs[irec,2]]; color=:black)
    end
    # display sources and receivers
    scatter!(possrcs[:,1], possrcs[:,2]; markershape=:star4, markersize=5, markercolor=:white, legend=nothing)
    scatter!(posrecs[:,1], posrecs[:,2]; markershape=:utriangle, markersize=4, markercolor=:white, legend=nothing)
    # display CPML boundaries
    plot!([halo*dx, lx-halo*dx], [halo*dy, halo*dy]; color=:white)
    plot!([halo*dx, lx-halo*dx], [ly-halo*dy, ly-halo*dy]; color=:white)
    plot!([halo*dx, halo*dx], [halo*dy, ly-halo*dy]; color=:white)
    plot!([lx-halo*dx, lx-halo*dx], [halo*dy, ly-halo*dy]; color=:white)
    plot!(; clims=(1000, 1300))
    display(p)

    savefig("shot$(i)_$(c0max).png")
end

In [ ]:
# Input parameters for acoustic simulation
boundcond = CPMLBoundaryConditionParameters(; halo=halo, rcoef=rcoef, freeboundtop=false)
params = InputParametersAcoustic(nt, dt, [nx, ny], [dh, dh], boundcond)

# Wave simulation builder
wavesim = build_wavesim(params; gradient=true, parall=:threads, check_freq=ceil(Int, sqrt(nt)))

# compute forward gaussian
with_logger(error_logger) do
    swforward!(wavesim, matprop_gauss, shots)
end

In [ ]:
# # Display seismograms for each shot
# for i in 1:nshots
#     p = plot(shots[i].recs.seismograms)
#     display(p)
# end
# # Check that seismograms are specular
# for i in 1:(nshots-1)÷2
#     @assert shots[i].recs.seismograms ≈ reverse(shots[nshots+1-i].recs.seismograms, dims=2)
# end

In [ ]:
# new receivers with observed seismograms
shots_obs = Vector{Shot{Float64}}()  #Pair{Sources, Receivers}}()
for i in 1:nshots
    # receivers definition
    recs = Receivers(copy(shots[i].recs.positions), nt; observed=copy(shots[i].recs.seismograms), invcov=Diagonal(ones(nt)))
    # add pair as shot
    push!(shots_obs, Shot(; srcs=shots[i].srcs, recs=recs)) # srcs => recs)
end

In [ ]:
# compute gradients and misfit
gradient, misfit = with_logger(error_logger) do
    swgradient!(wavesim, matprop_const, shots_obs; compute_misfit=true)
end
misfit

In [ ]:
p = plot_nice_heatmap_grad(gradient; lx=nx*dx, ly=ny*dy, step=dh)
corner = 250 ÷ dx
plot!([corner*dx, lx-corner*dx], [corner*dy, corner*dy]; color=:black)
plot!([corner*dx, lx-corner*dx], [ly-corner*dy, ly-corner*dy]; color=:black)
plot!([corner*dx, corner*dx], [corner*dy, ly-corner*dy]; color=:black)
plot!([lx-corner*dx, lx-corner*dx], [corner*dy, ly-corner*dy]; color=:black)
display(p)

p = plot_nice_heatmap_grad(gradient[corner:end-corner,corner:end-corner]; lx=(nx-corner*2)*dx, ly=(ny-corner*2)*dy, step=dh, shift=-dx*corner)
display(p)

savefig("gradient_$(c0max).png")

In [ ]:
using Optim

error_logger = ConsoleLogger(stderr, Logging.Warn)

function fg!(F,G,x)
    matprop = VpAcousticCDMaterialProperty(x)
    if G != nothing && F != nothing
        # compute gradients and misfit
        gradient, misfit = with_logger(error_logger) do
            swgradient!(wavesim, matprop, shots_obs; compute_misfit=true)
        end
        G .= gradient
        @show misfit
        return misfit
    elseif G != nothing
        # compute only gradient
        gradient = with_logger(error_logger) do
            swgradient!(wavesim, matprop, shots_obs)
        end
        G .= gradient
    elseif F != nothing
        # compute only misfit
        misfit = with_logger(error_logger) do
            swmisfit!(wavesim, matprop, shots_obs)
        end
        @show misfit
        return misfit
    end
  end

res = optimize(Optim.only_fg!(fg!), matprop_const.vp, Optim.LBFGS())

In [ ]:
sol = Optim.minimizer(res)

In [ ]:
display(plot_nice_heatmap(sol; lx=nx*dx, ly=ny*dy, step=dh))